### Segundo experimento de detecção de Alzermir com dois dataset

In [37]:
import os
import pandas as pd
import kagglehub
import io
from io import BytesIO
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torchvision import transforms, models
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import time
import csv  # Adicionando import faltante
import re

# Mapeia os rótulos numéricos para nomes das classes (apenas para referência posterior)
label_map = {
    0: 'Mild_Dementia',
    1: 'Moderate_Dementia',
    2: 'Non_Demented',
    3: 'Very_mild_Dementia'
}

# Faz o download da versão mais recente do dataset via kagglehub
path = kagglehub.dataset_download("borhanitrash/alzheimer-mri-disease-classification-dataset")

# Mapeia os nomes das pastas para rótulos numéricos
class_map = {
    'Mild Dementia': 0,
    'Moderate Dementia': 1,
    'Non Demented': 2,
    'Very mild Dementia': 3
}

# Cria um dataframe a partir das imagens contidas nas pastas
def criar_dataframe_de_imagens(base_path, class_map, source_name):
    dados = []
    for label_nome, label_id in class_map.items():
        pasta = os.path.join(base_path, label_nome)
        if not os.path.exists(pasta):
            continue
        for nome_arquivo in os.listdir(pasta):
            caminho_completo = os.path.join(pasta, nome_arquivo)
            dados.append({
                'image_path': caminho_completo,
                'label': label_id,
                'source': source_name
            })
    return pd.DataFrame(dados)

# Carrega os dados de imagem das pastas (OASIS)
df_train_oasis = criar_dataframe_de_imagens(
    '/kaggle/input/oasis-alzheimer-dataset/train', class_map, 'oasis_train')
df_test_oasis = criar_dataframe_de_imagens(
    '/kaggle/input/oasis-alzheimer-dataset/test', class_map, 'oasis_test')

# Carrega os dados do arquivo Parquet
df_parquet = pd.read_parquet('/kaggle/input/alzheimer-mri-disease-classification-dataset/Alzheimer MRI Disease Classification Dataset/Data/train-00000-of-00001-c08a401c53fe5312.parquet')

# Converte os bytes da imagem em objetos BytesIO (necessário para abrir com PIL)
df_parquet['image_path'] = df_parquet['image'].apply(lambda x: BytesIO(x['bytes']))

# Seleciona apenas as colunas necessárias e adiciona a coluna 'source'
df_parquet_final = df_parquet[['image_path', 'label']].copy()
df_parquet_final['source'] = 'parquet'

# Junta os três dataframes em um único dataframe geral
df_geral = pd.concat([df_parquet_final, df_train_oasis, df_test_oasis], ignore_index=True)

# Mostra a distribuição de amostras por fonte
print("\nDistribuição por fonte no df_geral:\n", df_geral['source'].value_counts())

# Cirar dataset customizado
class AlzheimerUnifiedDataset(Dataset):
  def __init__(self, dataframe, transform=None):
    self.df = dataframe
    self.transform = transform

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    entrada = self.df.iloc[idx]['image_path']
    label = self.df.iloc[idx]['label']

    # Se for caminho (string), abra a imagem
    if isinstance(entrada, str):
      image = Image.open(entrada).convert('RGB')
    else:
      image = Image.open(entrada).convert('RGB')

    if self.transform:
      image = self.transform(image)

    return image, label

# Split em treino, validação e teste
df_trainval, df_test = train_test_split(df_geral, test_size=0.15, stratify=df_geral['label'], random_state=42)
df_train, df_val = train_test_split(df_trainval, test_size=0.15, stratify=df_trainval['label'], random_state=42)

# Transforms padrão
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],    # padrão ImageNet
                        std=[0.229, 0.224, 0.225])
])

# Carregar os loaders
train_dataset = AlzheimerUnifiedDataset(df_train, transform=transform)
val_dataset = AlzheimerUnifiedDataset(df_val, transform=transform)
test_dataset = AlzheimerUnifiedDataset(df_test, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Modelo com AlexNet pré-treinado
model = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)

# Substitui a camada final para 4 classes
# A AlexNet tem um classifier com vários Layers
# Vamos modificar a útltima camada linear (classifier[6])
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 4)

# Colocar o modelo na GPU
model = model.to(device)

# Função de perda com pesos
weights = torch.tensor([1.0, 3.5, 1.0, 1.0]).to(device)
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Treinamento
num_epochs = 10
train_losses, val_losses = [], []

start_time = time.time()  # CORREÇÃO: star_time -> start_time

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    y_train_true, y_train_pred = [], []

    for images, labels in tqdm(train_loader, desc=f'Época {epoch + 1}/{num_epochs} - Treino'):
      images, labels = images.to(device), labels.to(device)
      optimizer.zero_grad()
      outputs = model(images)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()

      # Métricas treino
      _, preds = torch.max(outputs, 1)
      y_train_true.extend(labels.cpu().numpy())
      y_train_pred.extend(preds.cpu().numpy())

    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    train_accuracy = accuracy_score(y_train_true, y_train_pred)
    train_precision = precision_score(y_train_true, y_train_pred, average='weighted', zero_division=0)
    train_recall = recall_score(y_train_true, y_train_pred, average='weighted', zero_division=0)
    train_f1 = f1_score(y_train_true, y_train_pred, average='weighted', zero_division=0)

    # Validação
    model.eval()
    val_loss = 0.0
    y_val_true, y_val_pred = [], []

    with torch.no_grad():
      for images, labels in tqdm(val_loader, desc=f'Época {epoch+1}/{num_epochs} - Validação'):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()

        # Métricas validação
        _, preds = torch.max(outputs, 1)
        y_val_true.extend(labels.cpu().numpy())
        y_val_pred.extend(preds.cpu().numpy())
    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    val_accuracy = accuracy_score(y_val_true, y_val_pred)
    val_precision = precision_score(y_val_true, y_val_pred, average='weighted', zero_division=0)
    val_recall = recall_score(y_val_true, y_val_pred, average='weighted', zero_division=0)
    val_f1 = f1_score(y_val_true, y_val_pred, average='weighted', zero_division=0)
    print(f"""
    Época {epoch+1}/{num_epochs}
    ----------------------------
    Treino:
    Loss: {avg_train_loss:.4f}
    Acurácia: {train_accuracy:.4f}
    Precisão: {train_precision:.4f}
    Recall: {train_recall:.4f}
    F1 Score: {train_f1:.4f}
    Validação:
    Loss: {avg_val_loss:.4f}
    Acurácia: {val_accuracy:.4f}
    Precisão: {val_precision:.4f}
    Recall: {val_recall:.4f}
    F1 Score: {val_f1:.4f}
    """)

end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo de execução: {execution_time} segundos")

resultados = {
    "modelo": "AlexNet",
    "epocas": num_epochs,
    "tempo_total": round(execution_time, 2), # em segundos
    "acuracia": round(train_accuracy, 4),
    "precisao": round(train_precision, 4),
    "revocacao": round(train_recall, 4),
    "f1_macro": round(train_f1, 4)
}

def limpar_nome_arquivo(nome):
    return re.sub(r'[\/:"*?<>|]+', "_", nome)

nome_seguro = limpar_nome_arquivo("AlexNet")  # CORREÇÃO: usando string em vez de variável não definida

file_path = f"resultados_do_modelo_{nome_seguro}.csv"
campos = list(resultados.keys())

# Cria o arquivo se não existir
if not os.path.exists(file_path):
    with open(file_path, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=campos)
        writer.writeheader()
        writer.writerow(resultados)
else:
    # Apenas adiciona nova linha
    with open(file_path, 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=campos)
        writer.writerow(resultados)


Distribuição por fonte no df_geral:
 source
parquet    5120
Name: count, dtype: int64


Época 1/10 - Validação: 100%|██████████| 41/41 [00:01<00:00, 28.83it/s]



    Época 1/10
    ----------------------------
    Treino:
    Loss: 1.0122
    Acurácia: 0.5588
    Precisão: 0.5274
    Recall: 0.5588
    F1 Score: 0.5256
    Validação:
    Loss: 0.9425
    Acurácia: 0.5299
    Precisão: 0.6803
    Recall: 0.5299
    F1 Score: 0.4209
    


Época 2/10 - Validação: 100%|██████████| 41/41 [00:01<00:00, 22.96it/s]



    Época 2/10
    ----------------------------
    Treino:
    Loss: 0.8416
    Acurácia: 0.6310
    Precisão: 0.6272
    Recall: 0.6310
    F1 Score: 0.6228
    Validação:
    Loss: 0.8597
    Acurácia: 0.6187
    Precisão: 0.6516
    Recall: 0.6187
    F1 Score: 0.5907
    


...


    Época 8/10
    ----------------------------
    Treino:
    Loss: 0.1433
    Acurácia: 0.9448
    Precisão: 0.9450
    Recall: 0.9448
    F1 Score: 0.9449
    Validação:
    Loss: 0.2653
    Acurácia: 0.9020
    Precisão: 0.9043
    Recall: 0.9020
    F1 Score: 0.9003
    


Época 9/10 - Validação: 100%|██████████| 41/41 [00:02<00:00, 20.21it/s]



    Época 9/10
    ----------------------------
    Treino:
    Loss: 0.1258
    Acurácia: 0.9535
    Precisão: 0.9537
    Recall: 0.9535
    F1 Score: 0.9535
    Validação:
    Loss: 0.2298
    Acurácia: 0.9234
    Precisão: 0.9261
    Recall: 0.9234
    F1 Score: 0.9239
    


Época 10/10 - Validação: 100%|██████████| 41/41 [00:01<00:00, 28.70it/s]


    Época 10/10
    ----------------------------
    Treino:
    Loss: 0.0414
    Acurácia: 0.9851
    Precisão: 0.9852
    Recall: 0.9851
    F1 Score: 0.9851
    Validação:
    Loss: 0.3976
    Acurácia: 0.9142
    Precisão: 0.9178
    Recall: 0.9142
    F1 Score: 0.9131
    
Tempo de execução: 168.57792115211487 segundos
